In [54]:
from slack import WebClient
from slack.errors import SlackApiError
import requests
from newsapi import NewsApiClient
from datetime import datetime, timedelta

email_addresses = ['apacpress@tesla.com']

def extract_company_name(email):
    company_name = email.split('@')[-1].split('.')[0]
    if company_name:
        return company_name
    else:
        return None

def locate_google_news_alerts(emails, date = None):
   
    if date is None:
        date = (datetime.today()).strftime('%Y-%m-%d')
    date2 = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    api_key = '87d2b7f06cfa40ba8dde5bc7a635a7d5'
    newsapi = NewsApiClient(api_key=api_key)
    
    for email in emails:
        company_name = extract_company_name(email)
        if company_name:
            articles = newsapi.get_everything(q=company_name, language='en', from_param=date2, to=date)
            message = ''
            if articles['totalResults'] > 0:
                
                messsage = f"News alerts for {company_name} :"
                for article in articles['articles']:
                    message += f"Title: {article['title']}\nURL: {article['url']}\n"
                send_to_slack(message)    
                    
            else:
                send_to_slack(f"No news alerts found for {company_name} ")
        else:
            send_to_slack(f"Unable to extract company name from {email}")
def send_to_slack(message):
    
    client = WebClient(token='your_token')
    
    try:
        response = client.chat_postMessage(channel='your_channel', text=message)
        assert response["message"]["text"] == message
        print("News alerts sent to Slack successfully!")
    except SlackApiError as e:
        print(f"Error sending message to Slack: {e.response['error']}")   
def main():
     locate_google_news_alerts(email_addresses)  
if __name__ == "__main__":
    main()